In [63]:
import numpy as np

In [64]:
# Had a little help as always
# https://www.geeksforgeeks.org/dice-throw-problem/

# Here is the table of correct answers for 6 sided dice, to check accuracy
# https://wizardofodds.com/gambling/dice/2/
def find_ways(k,n,x):
    if x<1: return 0
    if n==1: return int(x<=k)#1 if true else 0
    num_ways = 0
    for i in range(1,k+1):
        num_ways += find_ways(k,n-1,x-i)
    return num_ways

In [65]:
find_ways(6,5,14)

540

In [66]:
'''
Dynamic Solution -- translated to numpy
'''
def findWays(k,n,x):
    # The extra row/col make indexing easier, but could be removed (it's not really too bad tho)
    table = np.zeros((n+1,x+1), dtype=np.uint64)
    
    if k < x:
        table[1][1:k+1] = 1
    else:
        table[1][1:x+1] = 1
    
    
    for i in range(2,n+1):
        for j in range(2,x+1):
            l = 1
            while l<j and l<=k:
                table[i][j] += table[i-1][j-l]
                l+=1

                
    print(table)
    return table[n][x]

    

In [67]:
# findWays(6,6,6)


In [68]:
def get_freq(k,n):
    x = n*k
    table = np.zeros((n+1,x+1), dtype = np.uint64)
    
    if k < x:
        table[1][:k+1] = 1
    else:
        table[1][:x+1] = 1
    
    
    for i in range(2,n+1):
        for j in range(2,x+1):
            #table[i][j] = table[i][j-1] + table[i-1][j-1]
            l = 1
            while l<j and l<=k:
                table[i][j] += table[i-1][j-l]
                l+=1
#    print(table)
    
    return table[n]

In [69]:
get_freq(6,3)

array([ 0,  0,  0,  1,  3,  6, 10, 15, 21, 25, 27, 27, 25, 21, 15, 10,  6,
        3,  1], dtype=uint64)

In [70]:
def get_list_freq(dice):
    #setup
    #there are n dice, like last time
    n = dice.size
    # x needs to be the upper bound, which would be rolling k on all dice
    x = np.sum(dice)
    # init the table
    table = np.zeros((n+1,x+1), dtype=np.uint64)
    
    if np.max(dice) < x:
        table[1][:np.max(dice)+1] = 1
    else:
        table[1][:x+1] = 1
    
    i = 1
    for k in dice:
        # we need to handle each dice face independently
        #might need temporary bound for 2nd loop
        for j in range(2,x+1):
            l = 1
            while l<j and l <=k:
                table[i][j] += table[i-1][j-l]
                l+=1   
        i += 1
#    print(table)
    return table[n]

In [71]:
print(get_list_freq(np.array([6,6,6])))
print(get_freq(6,3))

[ 0  0  0  1  3  6 10 15 21 25 27 27 25 21 15 10  6  3  1]
[ 0  0  0  1  3  6 10 15 21 25 27 27 25 21 15 10  6  3  1]


In [72]:
def get_dict_freq(some_dict):
    lis = []
    for key in some_dict:
        lis += [key]*some_dict[key]
    to_call = np.asarray(lis)
    # print(to_call)
    return get_list_freq(to_call)

In [73]:
test = {6:2, 4:1}
print(get_dict_freq(test))
print(get_list_freq(np.array([6,6,4])))

[ 0  0  0  1  3  6 10 14 18 20 20 18 14 10  6  3  1]
[ 0  0  0  1  3  6 10 14 18 20 20 18 14 10  6  3  1]


In [74]:
# np.random.choice(5000,200,replace=False)

In [101]:
'''
Attempting to use linear algebra to solve get_freq(6,2), then will build from there

size = nk-n+1
'''
size = 2*6-2+1


Y = np.ones((size,size))
Y = np.triu(Y)

Y = np.triu(np.flip(Y,1))

'''
I also need to initialize the starting array, a 1xSize arry with the initial frrequencies
'''

a = np.zeros(size)
a[0:6] = 1
print(a)

print(Y)

print(Y.T.dot(a))


[ 1.  1.  1.  1.  1.  1.  0.  0.  0.  0.  0.]
[[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.]
 [ 0.  0.  1.  1.  1.  1.  1.  1.  1.  0.  0.]
 [ 0.  0.  0.  1.  1.  1.  1.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  1.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
[ 1.  2.  3.  4.  5.  6.  5.  4.  3.  2.  1.]


In [152]:
def freq(k,n):
    size = k*n-n+1
    X = np.ones((size,size))
    X = np.triu(X)
    X = np.triu(np.flip(X,1))
    # print(X.T)
    a = np.zeros(size)
    a[0:k] = 1
    # print(a)
    return np.linalg.matrix_power(X.T,n-1).dot(a)

In [153]:
get_freq(6,3)

array([ 0,  0,  0,  1,  3,  6, 10, 15, 21, 25, 27, 27, 25, 21, 15, 10,  6,
        3,  1], dtype=uint64)

In [154]:
freq(6,2)

array([ 1.,  2.,  3.,  4.,  5.,  6.,  5.,  4.,  3.,  2.,  1.])

In [160]:
D = np.ones((11,11))
D = np.triu(D)
D = np.triu(np.flip(D,1))
b = np.zeros(11)
b[0:6] = 1

print(np.linalg.matrix_power(D.T,1).dot(b))



[ 1.  2.  3.  4.  5.  6.  5.  4.  3.  2.  1.]
